In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 

city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,kapaa,US,22.0752,-159.3190,77.32,87,100,14.97,light rain
1,1,saint-pierre,RE,-21.3393,55.4781,71.28,71,75,4.61,broken clouds
2,2,trofors,NO,65.5333,13.4063,53.47,88,100,4.72,light rain
3,3,chardara,KZ,41.2547,67.9692,87.67,20,0,14.65,clear sky
4,4,babu,CN,24.4167,111.5167,81.61,75,99,2.46,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temp you want?"))
max_temp = float(input("What is the maximum temp you wnat?"))


What is the minimum temp you want?50
What is the maximum temp you wnat?65


In [21]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,trofors,NO,65.5333,13.4063,53.47,88,100,4.72,light rain
8,8,yar-sale,RU,66.8333,70.8333,56.88,63,100,11.10,overcast clouds
11,11,talnakh,RU,69.4865,88.3972,58.21,63,100,3.42,overcast clouds
16,16,bredasdorp,ZA,-34.5322,20.0403,58.57,68,85,8.72,overcast clouds
17,17,akureyri,IS,65.6835,-18.0878,53.53,66,40,17.27,scattered clouds
20,20,cape town,ZA,-33.9258,18.4232,64.38,71,75,5.75,broken clouds
21,21,aberdeen,GB,57.1437,-2.0981,64.65,82,75,6.91,broken clouds
23,23,rio grande,BR,-32.0350,-52.0986,59.11,93,99,14.56,overcast clouds
32,32,nuuk,GL,64.1835,-51.7216,54.21,62,40,8.05,scattered clouds
36,36,nikolskoye,RU,59.7035,30.7861,60.80,88,74,6.62,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.count()

Unnamed: 0             164
City                   164
Country                163
Lat                    164
Lng                    164
Max Temp               164
Humidity               164
Cloudiness             164
Wind Speed             164
Current Description    164
dtype: int64

In [24]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df.dropna()
clean_df

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,trofors,NO,65.5333,13.4063,53.47,88,100,4.72,light rain
8,8,yar-sale,RU,66.8333,70.8333,56.88,63,100,11.10,overcast clouds
11,11,talnakh,RU,69.4865,88.3972,58.21,63,100,3.42,overcast clouds
16,16,bredasdorp,ZA,-34.5322,20.0403,58.57,68,85,8.72,overcast clouds
17,17,akureyri,IS,65.6835,-18.0878,53.53,66,40,17.27,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
653,653,amparafaravola,MG,-17.5833,48.2167,58.59,90,70,5.73,broken clouds
654,654,tolaga bay,NZ,-38.3667,178.3000,53.19,94,100,10.69,overcast clouds
657,657,charlestown,AU,-32.9500,151.6667,52.74,79,52,5.99,broken clouds
678,678,high level,CA,58.5169,-117.1360,60.91,63,0,5.75,clear sky


In [25]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,trofors,NO,53.47,light rain,65.5333,13.4063,
8,yar-sale,RU,56.88,overcast clouds,66.8333,70.8333,
11,talnakh,RU,58.21,overcast clouds,69.4865,88.3972,
16,bredasdorp,ZA,58.57,overcast clouds,-34.5322,20.0403,
17,akureyri,IS,53.53,scattered clouds,65.6835,-18.0878,
20,cape town,ZA,64.38,broken clouds,-33.9258,18.4232,
21,aberdeen,GB,64.65,broken clouds,57.1437,-2.0981,
23,rio grande,BR,59.11,overcast clouds,-32.0350,-52.0986,
32,nuuk,GL,54.21,scattered clouds,64.1835,-51.7216,
36,nikolskoye,RU,60.80,broken clouds,59.7035,30.7861,


In [26]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    # get the lat and lon
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame

    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print ("skip city")
    
        

skip city
skip city
skip city
skip city
skip city
skip city
skip city
skip city
skip city
skip city
skip city
skip city
skip city
skip city
skip city
skip city
skip city
skip city
skip city
skip city
skip city
skip city


In [27]:
# 7. Drop the rows where there is no Hotel Name.

hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,trofors,NO,53.47,light rain,65.5333,13.4063,Utpå Neset AS
8,yar-sale,RU,56.88,overcast clouds,66.8333,70.8333,Yalemd
11,talnakh,RU,58.21,overcast clouds,69.4865,88.3972,Talnakh Hotel
16,bredasdorp,ZA,58.57,overcast clouds,-34.5322,20.0403,Bredasdorp Country Manor
17,akureyri,IS,53.53,scattered clouds,65.6835,-18.0878,Icelandair Hotel Akureyri
...,...,...,...,...,...,...,...
653,amparafaravola,MG,58.59,broken clouds,-17.5833,48.2167,Aina Franco
654,tolaga bay,NZ,53.19,overcast clouds,-38.3667,178.3000,Tolaga Bay Motel
657,charlestown,AU,52.74,broken clouds,-32.9500,151.6667,Panorama Motor Inn Hotel
678,high level,CA,60.91,clear sky,58.5169,-117.1360,Days Inn by Wyndham High Level


In [35]:
# 8a. Create the output File (CSV)

output_data_file = "Weather_Database/WeatherPy_vacation.csv.csv"

# 8b. Export the City_Data into a csv
clean_df.to_csv(output_data_file, index_label="City_ID")

In [33]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_df[["Lat", "Lng"]]

In [41]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level = 1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure

fig

import matplotlib.pyplot as plt

plt.savefig ("Weather_Database/WeatherPy_vacation_map.png")


<Figure size 432x288 with 0 Axes>